# Nuclei profiling - module introduction

Unstable release - test notebook to showcase modules of the framework. The framework is first and foremost designed for easy graphical representation of the nuclear structure, using the Plotly Python module. Additional features such as the projection of the nucleon density can also be extracted. Additional features might be added on occasions. This showcase "book" outlines some of the basic properties of the python module.<br> <br>
Default renderer uses 'iframe' -> saves figures in HTML in folder "iframe_figures" (best performance but remember to clear folder occasionally)

In [1]:
# Plotting tools
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "iframe"       #'iframe' or 'notebook' or 'colab' or 'jupyterlab' 'browser'
# Math tools
import numpy as np

In [2]:
# The magic
from nuclear_profile import NuclearProfile

# Shape enginering with spherical harmonics

Most atomic nuclei exhibit some sort of intrinsic deformation due to the multipole moments of the nucleon distribution function. Consequently, the nuclear structure can be of very complex forms which are not easily modeled. Though, experts in the field have come up with clever ways of modeling these structures in terms of the spherical harmonics, which often are known for their relation to the electron orbitals. These shape variations can be modeled by an equation for the radius $R(\theta, \varphi)$ that utilizes these harmonics, where each corresponds to the magnitude of the multipole moment 

$$R'(\theta, \phi)= R_0 (1 + \beta_2 (\cos(\gamma) Y_2^0(\theta) + \frac{1}{\sqrt(2)}\sin(\gamma) Y_2^2(\theta, \phi)) + \beta_3Y_3^0 + \beta_4Y_4^0) $$

Here $R_0$ denotes the rms radius of the nuclei, $\beta_n$ the n´th multipole moment of the nucleon distribution, $\gamma$ the angular asymmetry, and $Y_{\ell}^m(\theta, \phi)$ denotes the spherical harmonics. For a perfect sphere $\beta_2=\beta_3=\beta_4=0$, the resulting surface radius would be constant $R'(\theta, \phi)= R_0$

### The wood-saxon density profile

The Wood-Saxon profile is a function commonly used to describe the nuclear density distribution of nuclei. It provides an approximate representation of how nucleons (protons and neutrons) are distributed within the nucleus. The Wood-Saxon profile can account for the effects of nuclear shape deformation on the nucleon density by using the equation for the deformed surface, as outlined above.

$$\large\rho (\theta, \phi)=   \frac{\rho_0}{1+\exp\left(\frac{r-R'(\theta, \phi)}{a_0}\right)} $$

#### <u>Constructor

Each nuclei are quickly initilized through their constructor ***NuclearProfile(R0, a0, beta2=0, gamma=0, beta3=0, beta4=0)***. The most basic information needed are the nuclear radius $R_0$ and the deffusion $a_0$. The deformation can further be changed through the class methods ***set_beta2(..)***, ***set_beta3(..)*** and ***set_beta4(..)***

## Quadrupole deformation

A small example is to illustrate the nuclear shape with quadrupole deformation $\beta_2$, and triaxiality $\gamma$ is provided. The example illustrate the possible difference in shape for nuclei with same quadrupole deformation.

In [3]:
# Initilize nuclei with unit radius and arbitrary deffusion, varying the triaxiality
Prolate  = NuclearProfile(R0=1., a0=0.1, beta2=0.25, gamma=0)
TriAxial = NuclearProfile(R0=1., a0=0.1, beta2=0.25, gamma=30)
Oblate   = NuclearProfile(R0=1., a0=0.1, beta2=0.25, gamma=60)
quadrupoles = [Prolate, TriAxial, Oblate]

In [4]:
# prepare figure to hold nuclei surfaces
fig = make_subplots(rows=1, cols=3,
                    specs=[[{'is_3d': True}, {'is_3d': True},  {'is_3d': True}]],
                    subplot_titles=['Prolate', 'Tri-Axial', 'Oblate'],
                   )

for idx, profile, in enumerate(quadrupoles):
    Rx, Ry, Rz = profile.GetSurface()          # Get the nuclie surface
    Wx, Wy, Wz = profile.GetWireFrame()        # Get the wireframe (mainly for nice plotting)
    
    fig.add_trace(go.Surface(x=Rx, y=Ry, z=Rz, surfacecolor=Rx**2 + Ry**2 + Rz**2, showscale=False, colorscale='Plasma'), 1, idx+1)
    fig.add_trace(go.Scatter3d(x=Wx, y=Wy, z=Wz, mode='lines', line_width=1, line_color='rgba(10,10,10, 1)'), 1, idx+1)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
fig.update_layout(showlegend=False)
fig.show()

# Nuclear diffusion

The nuclear defusion describes the rate of density falloff, as the distance from the center increases. This feature can be illustrated by getting the complete density grid for the nuclei and plotting it in a volume plot. The examples show two types of diffusion known as skin-like and halo-like.

In [5]:
SkinLike = NuclearProfile(R0=1., beta2=0.18, gamma=27, a0=0.075)
HaloLike = NuclearProfile(R0=1., beta2=0.18, gamma=27, a0=0.3)
skintypes = [SkinLike, HaloLike]

In [6]:
scount=[20, 30]
sopacity=[0.1, 0.1]
fig = make_subplots(rows=1, cols=2,
                    specs=[[{'is_3d': True}, {'is_3d': True}]],
                    subplot_titles=['Skin like', 'Halo like'], horizontal_spacing = 0.01,vertical_spacing = 0.01)

for idx, profile, in enumerate(skintypes):
    Rhox, Rhoy, Rhoz, density = profile.GetDensityGrid()     # Get the density at each point in space
    fig.add_trace(go.Volume(x=Rhox.flatten(), y=Rhoy.flatten(), z=Rhoz.flatten(), value=density.flatten(),
    isomin=0.05, isomax=0.4, opacity=sopacity[idx], surface_count=scount[idx], showscale=False), 1, idx+1)

fig.show()

# Plane projection of density profile

The xy projection can be obtained by NuclearProfile.GetProjection("XY"). Likewise, the projections can also be obtained in the "YZ" and "ZX" plane, the order of the letters does not matter, ie. "XY"="YX". 

In [7]:
nuclei = NuclearProfile(R0=1., a0=0.15, beta2=0.20, gamma=30)
projections = ["XY", "XZ", "YZ"]

In [8]:
fig = make_subplots(rows=2, cols=3, subplot_titles=['XY-Projection', 'XZ-Projection', 'YZ-Projection'],
                   horizontal_spacing = 0.05, vertical_spacing=0.05)

for idx, iprojection, in enumerate(projections):
    xaxis, yaxis, values = nuclei.GetProjection(iprojection)     # .GetProjection()
    fig.add_trace(go.Heatmap(x=xaxis, y=yaxis, z=values, showscale=False, connectgaps=True, zsmooth='best'), 1, idx+1)
    fig.add_trace(go.Contour(x=xaxis, y=yaxis, z=values,contours_coloring='heatmap', contours=dict(coloring ='heatmap', showlabels=True), showscale=False), 2, idx+1)
    fig.update_layout(height=650, width=1000, showlegend=False, title_text="")
fig.show()